# AnswerALS Cloud: Technical Requirements

#### Alex LeNail, alex@lenail.org

This notebook describes requirements for a system to **"Query by Data"** and **"Compute on Data"** against the AnswerALS datastore in Azure. 

[This document provides background on the problem, and a proposed technical architecture to solve it](https://docs.google.com/document/d/1nPZIVqYmNdhf-OSJI7oaNlwPq334o-h0EWiNTRKwjzs/edit?usp=sharing). One section from that document I'd like to highligh here is the conclusion, regarding considerations for the system: 

> We should evaluate competing options on the basis of: 
> - **Query speed**: How long would it take to run a typical “data query” via each of these technologies? 
> - **Flexibility & Completeness**: What operations are supported by each of these technologies? For Compute on Data specifically: Could I run python, R, arbitrary binaries / bash? What would the technology allow me? 
> - **Familiarity & ease of use**: Are the end users already familiar with the proposed technology? If not, what is the learning curve for an average bioinformatician? Once they’ve learned, how easy is it for an experienced user to run queries? 
> - **Development Cost**: Whatever we develop should rely heavily on Azure technologies, and avoid reinventing the wheel. 
> - **Maintenance Cost**: Whatever we develop should be sufficiently robust and minimal as to require minimal maintenance, and it should be sufficiently simple that a grad student would be able to debug it, should they need to. 

This notebook contains a set of representative operations we want to execute via this system. I'm calling the system "QBD" for "Query By Data"

Once all of the cells in this notebook run and yeild correct answers, then it can be said that our technical requiremnts for the system were met. We make no hard constraints on the implementation, but we hope implementers will heed the 5 considerations described above. 



In [1]:
import numpy as np
import pandas as pd
import requests
# import qbd

The python package `qbd` provides a python interface to the QBD API. 

Consortium users will authenticate using their credentials, granting them a token valid for ~24h. 

In [ ]:
session_token = qbd.get_session(email='lenail@mit.edu', password="XXXXXX")

# I. 

### 1. Find male ALS patients over the age of 50

Possible via CDR

In [2]:
patients = pd.read_json('http://thecdr.westus.cloudapp.azure.com:3000/patient?sex=eq.Male&Condition=eq.ALS&age=gt.50').NeuroGUID.tolist()
print(len(patients), 'patients meet those criteria')

352 patients meet those criteria


### 2. Get their gVCFs

Possible via CDR

In [3]:
files = pd.read_json(f'http://thecdr.westus.cloudapp.azure.com:3000/file?omic=eq.genomics&data_level=eq.3&NeuroGUID=in.({",".join(patients)})')
files = files[files.path.str.endswith('.raw.vcf.gz.tbi') | files.path.str.endswith('.raw.vcf.gz')]
print('Of those, we have gVCFs for', int(len(files)/2))
files.head()


Of those, we have gVCFs for 169


,CGND_ID,NeuroGUID,data_level,differentiation,experiment,iPSC_Line,line,omic,path
5,CGND-HDA-01381,NEURR881FKY,3,NaN,WGS,NaN,NaN,genomics,genomics/3_vcf/CGND_12620/Sample_CGND-HDA-0138...
8,CGND-HDA-01381,NEURR881FKY,3,NaN,WGS,NaN,NaN,genomics,genomics/3_vcf/CGND_12620/Sample_CGND-HDA-0138...
13,CGND-HDA-01376,NEUDH063DEA,3,NaN,WGS,NaN,NaN,genomics,genomics/3_vcf/CGND_12620/Sample_CGND-HDA-0137...
15,CGND-HDA-01376,NEUDH063DEA,3,NaN,WGS,NaN,NaN,genomics,genomics/3_vcf/CGND_12620/Sample_CGND-HDA-0137...
26,CGND-HDA-01371,NEUCT842RJV,3,NaN,WGS,NaN,NaN,genomics,genomics/3_vcf/CGND_12620/Sample_CGND-HDA-0137...


### 3. Which of them have variants in SOD1? 

<span style="color:darkred">Required from QBD</span>

In [15]:
gVCFs_and_indexes = [paths.tolist() for GUID, paths in files.groupby('NeuroGUID')['path']]

def mapper(gVCF_and_index, **kwargs):
#     This function takes a gVCF file, and using various python genomics packages
#     determines whether the gVCF contains one or more variants at the SOD1 locus

results = qbd.map(mapper, gVCFs_and_indexes, interpreter='python', args={'reference_genome': '/data/hg38.gtf'}, session=session_token)

The above shows an example use case for the principal function provided by QBD: `map`. 
    
In the example use case, the AnswerALS member analyst defines a function `mapper` which takes a single file and keyword arguments (`kwargs`)

This function is then applied against all the files in the list `gVCFs_and_indexes`, and the results are returned in a python list.



### 4. Compare the RNA expression of SOD2 of patients with SOD1 mutations versus not

<span style="color:darkred">Required from QBD</span>

In [ ]:
def mapper(gVCF_and_index, **kwargs):
#     This function takes a gVCF file, and using various python genomics packages,
#     counts the variants which fall in each of the three types of regions.

results = qbd.map(mapper, gVCFs_and_indexes, interpreter='python', args={'reference_genome': '/data/hg38.gtf'}, session=session_token)

In [3]:
# plot the results (locally -- without qbd)

# II. 

### 1. Get female patients / female controls // male patients / male controls

Possible via CDR

In [4]:
female_patients = pd.read_json('http://thecdr.westus.cloudapp.azure.com:3000/patient?sex=eq.Female&Condition=eq.ALS').NeuroGUID.tolist()
female_controls = pd.read_json('http://thecdr.westus.cloudapp.azure.com:3000/patient?sex=eq.Female&Condition=eq.Healthy%20Control').NeuroGUID.tolist()

male_patients = pd.read_json('http://thecdr.westus.cloudapp.azure.com:3000/patient?sex=eq.Male&Condition=eq.ALS').NeuroGUID.tolist()
male_controls = pd.read_json('http://thecdr.westus.cloudapp.azure.com:3000/patient?sex=eq.Male&Condition=eq.Healthy%20Control').NeuroGUID.tolist()

len(female_patients), len(female_controls), len(male_patients), len(male_controls)

(262, 63, 454, 33)

### 2. What are differentially expressed proteins between female/female vs male/male? 

<span style="color:darkred">Required from QBD</span>

In [ ]:
def mapper(gVCF_and_index, **kwargs):
#     This function takes a gVCF file, and using various python genomics packages,
#     counts the variants which fall in each of the three types of regions.

results = qbd.map(mapper, gVCFs_and_indexes, interpreter='python', args={'reference_genome': '/data/hg38.gtf'}, session=session_token)

# III. 

### 1. Get the gVCF files of the entire cohort

Possible via CDR

In [5]:
files = pd.read_json('http://thecdr.westus.cloudapp.azure.com:3000/file?omic=eq.genomics&data_level=eq.3')
files = files[files.path.str.endswith('.raw.vcf.gz.tbi') | files.path.str.endswith('.raw.vcf.gz')]
gVCFs_and_indexes = [paths.tolist() for GUID, paths in files.groupby('NeuroGUID')['path']]
files.head()

,CGND_ID,NeuroGUID,data_level,differentiation,experiment,iPSC_Line,line,omic,path
5,CGND-HDA-01381,NEURR881FKY,3,NaN,WGS,NaN,NaN,genomics,genomics/3_vcf/CGND_12620/Sample_CGND-HDA-0138...
8,CGND-HDA-01381,NEURR881FKY,3,NaN,WGS,NaN,NaN,genomics,genomics/3_vcf/CGND_12620/Sample_CGND-HDA-0138...
13,CGND-HDA-01376,NEUDH063DEA,3,NaN,WGS,NaN,NaN,genomics,genomics/3_vcf/CGND_12620/Sample_CGND-HDA-0137...
15,CGND-HDA-01376,NEUDH063DEA,3,NaN,WGS,NaN,NaN,genomics,genomics/3_vcf/CGND_12620/Sample_CGND-HDA-0137...
26,CGND-HDA-01371,NEUCT842RJV,3,NaN,WGS,NaN,NaN,genomics,genomics/3_vcf/CGND_12620/Sample_CGND-HDA-0137...


### 2. What is the average number of variants per patient in our  cohort? 

<span style="color:darkred">Required from QBD</span>

In [ ]:
def mapper(gVCF_and_index, **kwargs):
#     This function takes a gVCF file, and using various python genomics packages,
#     determines the number of variants contained in the gVCF.

results = qbd.map(mapper, gVCFs_and_indexes, interpreter='python', args={}, session=session_token)

### 3. How many of each patients' variants are exonic, intronic, and intergenic, per the reference genome? 

<span style="color:darkred">Required from QBD</span>

In [ ]:
def mapper(gVCF_and_index, **kwargs):
#     This function takes a gVCF file, and using various python genomics packages,
#     counts the variants which fall in each of the three types of regions.

results = qbd.map(mapper, gVCFs_and_indexes, interpreter='python', args={'reference_genome': '/data/hg38.gtf'}, session=session_token)

### 4. Get the ATAC-Seq peaks files for every patient in the cohort for which we have peaks

Possible via CDR

In [6]:
peaks_files = pd.read_json('http://thecdr.westus.cloudapp.azure.com:3000/file?omic=eq.epigenomics&data_level=eq.3')
peaks_files.head()

""


### 5. What distribution of variants fall in peaks versus not for each patient? 

<span style="color:darkred">Required from QBD</span>

In [ ]:
def mapper(gVCF_and_index, **kwargs):
#     This function takes a gVCF file, and using various python genomics packages,
#     counts the variants which fall in peaks versus not.

results = qbd.map(mapper, gVCFs_and_indexes, interpreter='python', args={'reference_genome': '/data/hg38.gtf'}, session=session_token)

# IV.

more ideas? 

# QBD API Design -- Draft

## `qbd.get_session`


#### Arguments: 
- `email` (string):
- `password` (string):

#### Returns: 
- `str`:


## `qbd.map`


#### Arguments: 
- `mapper` (python function, or filepath string): either a python function or a path to a bash or R script.
- `files` (list): a python list of filepaths, either on Azure or the local filesystem.
- `interpreter` (string): ['python', 'R', or 'shell']
- `args` (dict): a dictionary of keyword parameters and associated values, to be passed to the mapper. This dictionary may contain paths to reference datasets (either on the client or in the cloud) which must be appropriately uploaded to each of the machines executing the job. 

#### Returns: 
- `list`: the output of the `mapper` against each of the `files`. The mapper may write files to disk on Azure, in which case this list is a list of cloud filepaths. 
